In [1]:
# Dependencies
import requests
import json
import pandas as pd
import time
from geopy.geocoders import GoogleV3

# Google developer API key
from config import gkey

sorted_agencies_path = 'agencies_df_places_4_27_21.csv'
sorted_df = pd.read_csv(sorted_agencies_path)
sorted_df



,Unnamed: 0,agency_name,agency_status,agency_address,agency_latitude,agency_longitude
0,0,Roger Gilbert - Prudential Financial,OPERATIONAL,"180 East Dykes Street, Cochran",32.388932,-83.351960
1,1,Rutherford Insurance,OPERATIONAL,"401 North Grant Street, Fitzgerald",31.718610,-83.252750
2,2,Global-Net Insurance,OPERATIONAL,"5305 Anson Avenue, Eastman",32.200026,-83.176214
3,3,Derek Hutcheson - State Farm Insurance Agent,OPERATIONAL,"401 Main Street, Eastman",32.195999,-83.174382
4,4,Nationwide Insurance: Steve Allen Martin,OPERATIONAL,"207 South Grant Street, Fitzgerald",31.713272,-83.253073
...,...,...,...,...,...,...
1825,29994,Michele Green - COUNTRY Financial representative,OPERATIONAL,"187 South Davis Road, LaGrange",33.052274,-84.984456
1826,30051,Walt McAbee - State Farm Insurance Agent,OPERATIONAL,"315 Thomaston Street, Zebulon",33.103893,-84.342495
1827,30070,John E Carter III - Aflac Insurance Agent,OPERATIONAL,"313 Troop Drive, Fortson",32.613742,-84.994139
1828,30336,Alfa Insurance - Lazaro Valle Agency,OPERATIONAL,"2120 Bellevue Road, Dublin",32.531313,-82.940303


In [2]:
# set the pause duration between api requests
pause = 0.1

In [3]:
# create a column to put lat-long into the format google likes - this just makes it easier to call their API
sorted_df['latlng'] = sorted_df.apply(lambda row: '{},{}'.format(row['agency_latitude'], row['agency_longitude']), axis=1)
sorted_df

,Unnamed: 0,agency_name,agency_status,agency_address,agency_latitude,agency_longitude,latlng
0,0,Roger Gilbert - Prudential Financial,OPERATIONAL,"180 East Dykes Street, Cochran",32.388932,-83.351960,"32.3889322,-83.35195999999998"
1,1,Rutherford Insurance,OPERATIONAL,"401 North Grant Street, Fitzgerald",31.718610,-83.252750,"31.718610100000003,-83.25274999999998"
2,2,Global-Net Insurance,OPERATIONAL,"5305 Anson Avenue, Eastman",32.200026,-83.176214,"32.20002589999999,-83.17621390000001"
3,3,Derek Hutcheson - State Farm Insurance Agent,OPERATIONAL,"401 Main Street, Eastman",32.195999,-83.174382,"32.1959994,-83.174382"
4,4,Nationwide Insurance: Steve Allen Martin,OPERATIONAL,"207 South Grant Street, Fitzgerald",31.713272,-83.253073,"31.7132719,-83.2530731"
...,...,...,...,...,...,...,...
1825,29994,Michele Green - COUNTRY Financial representative,OPERATIONAL,"187 South Davis Road, LaGrange",33.052274,-84.984456,"33.052274,-84.98445600000001"
1826,30051,Walt McAbee - State Farm Insurance Agent,OPERATIONAL,"315 Thomaston Street, Zebulon",33.103893,-84.342495,"33.103893,-84.3424948"
1827,30070,John E Carter III - Aflac Insurance Agent,OPERATIONAL,"313 Troop Drive, Fortson",32.613742,-84.994139,"32.613742200000004,-84.99413940000002"
1828,30336,Alfa Insurance - Lazaro Valle Agency,OPERATIONAL,"2120 Bellevue Road, Dublin",32.531313,-82.940303,"32.5313131,-82.9403035"


In [5]:
# pass the Google API latlng data to reverse geocode it
def reverse_geocode(latlng):
    time.sleep(pause)
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}&key={}'
    request = url.format(latlng, gkey)
    response = requests.get(request)
    data = response.json()
    if len(data['results']) > 0:
        return data['results'][0] #if we got results, return the first result
    
geocode_results = sorted_df['latlng'].map(reverse_geocode)


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
def get_city(geocode_result):
     if 'address_components' in geocode_result:
        for address_component in geocode_result['address_components']:
            if 'locality' in address_component['types']:
                return address_component['long_name']
            
def get_state(geocode_result):
     if 'address_components' in geocode_result:
        for address_component in geocode_result['address_components']:
            if 'administrative_area_level_1' in address_component['types']:
                return address_component['long_name']
            
def get_county(geocode_result):
     if 'address_components' in geocode_result:
        for address_component in geocode_result['address_components']:
            if 'administrative_area_level_2' in address_component['types']:
                return address_component['long_name']

In [ ]:
sorted_df['city'] = geocode_results.map(get_city)                
sorted_df['state'] = geocode_results.map(get_state)
sorted_df['county'] = geocode_results.map(get_county)
sorted_df

In [ ]:
sorted_df.to_csv(r'C:\Users\erinz\Desktop')